# Spoken language detection

## 1. 

In [1]:
import os
import random
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
import librosa
import matplotlib.pyplot as plt
import pandas as pd
import IPython

2024-05-22 20:22:14.438059: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 20:22:14.441024: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 20:22:14.479372: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 20:22:15.213897: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
PL_DIR = os.path.join('languages_audio', 'languages_audio_wav', 'pl', 'clips')
PL_DIR

In [ ]:
# create tensorflow dataset
pl_dataset = tf.data.Dataset.list_files(PL_DIR + '\*.wav')
pl_dataset

In [ ]:
pl_dataset.as_numpy_iterator().next()

In [ ]:
pl_dataset.as_numpy_iterator().next()

In [ ]:
df = pd.read_csv(os.path.join('languages_audio', 'languages_audio_mp3', 'pl', 'validated.tsv'), sep='\t')
df

In [ ]:
df[df['gender'] == 'female_feminine']

In [ ]:
df[df['gender'] == 'male_masculine']

In [ ]:
df.groupby('sentence_id').count()

In [ ]:
df['sentence_id'].value_counts()

In [ ]:
df['sentence_id'].nunique()

In [ ]:
df['client_id'].value_counts().sort_values(ascending=True)

In [ ]:
df[['sentence_id', 'client_id']].groupby(by='sentence_id').count().sort_values(by='client_id', ascending=False)


In [ ]:
df[['client_id', 'sentence_id']].sort_values(by='client_id', ascending=False)

In [ ]:
df[['client_id', 'sentence_id']].sort_values(by='client_id', ascending=False)

In [ ]:
df[df['client_id'] == 'ffffc00a5ed69f08a486837064ec2caeff21fe06264c3dd733f633fb6c2ae9aeb561a5a6f43e000554d4b3cc171644ba4ce971177af1cb54f9bd8cc153e71a5c']

In [ ]:
# # sprawdzenie czy client_id oraz sequence_id się nie duplikują (czy jeden klient ma rózne sekwencej)
# 
# for client_id in df['client_id'].unique():
#     sentence_count = df['sentence_id'][df['client_id']  == client_id].count()
#     client_id_count = df['client_id'][df['client_id']  == client_id].value_counts().iloc[0]
#     if sentence_count != client_id_count:
#         print(True)


In [ ]:
df['sentence_id'][df['client_id']  == 'ffffc00a5ed69f08a486837064ec2caeff21fe06264c3dd733f633fb6c2ae9aeb561a5a6f43e000554d4b3cc171644ba4ce971177af1cb54f9bd8cc153e71a5c'].count()

In [ ]:
df['client_id'][df['client_id']  == 'ffffc00a5ed69f08a486837064ec2caeff21fe06264c3dd733f633fb6c2ae9aeb561a5a6f43e000554d4b3cc171644ba4ce971177af1cb54f9bd8cc153e71a5c'].value_counts()

In [ ]:
# TODO: WZiąć plik validated.tsv nastepnie podzielić go na mężczyzn i kobiety (dwa osobne dataset). następnie wrzucać do każdego ze zbiorów (najpiew testwoy, walidacyjny, treningowy) client_id od najmniejszej liczby wystąpień w obydwu  dopasować czas trwania każdej próbki (najpierw obcinać od środka, potem dopełniać zerami), zmienić na spektogram i dopiero wtedy dodać (koniec funkcji). druga funkcja będzie jako augumentowanie losowych danych (losowo wybiera funkcję która coś zrobi z danymi) jeżeli moje zbiory kobiet i mężczyzn nie będą spełniały wymaganej ilości próbek którą się poda. Na wyjściu ma być tensor z próbkami spektogramu. To zrobić finalnie dla każdego folderu (języka). Wtedy dopiero dodać label w formie one hot encoding

## Load data:

In [13]:
df = pd.read_csv(os.path.join('..', '..', 'languages', 'lt', 'validated.tsv'), sep='\t', usecols=['client_id', 'path', 'sentence_id', 'gender', 'locale'])

## Change mp3 to wav name

In [14]:
# df = df.apply(lambda path: path.str.replace('.mp3', '.wav'))

## Split to woman and man sets

In [15]:
df

,client_id,path,sentence_id,gender,locale
0,05264b41636056b4c082890776306d78a64153a17761b0...,common_voice_lt_25188648.mp3,116a9dc506e249a0640eba08aef91cbe456d1c376566e0...,NaN,lt
1,14aa994e528fe04b837f68f9bc5c6268a3a3e251496eb6...,common_voice_lt_25125425.mp3,0aa7267734feda11b88b58076c6650f721ae067c99ac2f...,NaN,lt
2,40c8016380d4f3b6b6925bbb61e503513e9b6180a2d000...,common_voice_lt_25209338.mp3,14ae0dafd4ce037b354ad30b910cd3a11bf7c1d7782121...,NaN,lt
3,5e0759b675e2e5eac5dda7e93e2c63cda8c508cdb6ee8b...,common_voice_lt_26717831.mp3,1aad0fa17da428819d8495eabf5ac5f522674bb82364c5...,NaN,lt
4,6259e29a1089285f79483569c1accf7d6d83217709a50a...,common_voice_lt_26716253.mp3,1aef8e56b5672585da41597ce24d2b155287c5dc89cbf1...,NaN,lt
...,...,...,...,...,...
16638,e34db1479bd223145489573ff58e246c8b30e9de09bdc5...,common_voice_lt_25162613.mp3,10ef89654fb72a7b7b6d45deb809fb71216208a3f0f1ed...,male_masculine,lt
16639,e34db1479bd223145489573ff58e246c8b30e9de09bdc5...,common_voice_lt_25162614.mp3,10db81bb49e598d5e22d2c8fafea545691d93d08fa25cf...,male_masculine,lt
16640,e34db1479bd223145489573ff58e246c8b30e9de09bdc5...,common_voice_lt_25162615.mp3,1168759dfcaf33a91d6a0afbe2379542c1c2e5f04d4d42...,male_masculine,lt
16641,e34db1479bd223145489573ff58e246c8b30e9de09bdc5...,common_voice_lt_25162616.mp3,1141867eb08255f5a2098121972fa6b3c8b3bc9b1807eb...,male_masculine,lt


In [16]:
woman_filter = df['gender'] == 'female_feminine'
man_filter = df['gender'] == 'male_masculine'

In [17]:
df_women = df[woman_filter]
df_women

,client_id,path,sentence_id,gender,locale
100,cb07d781da89d6a623a37c56addb82b9dfc8fd7ced0339...,common_voice_lt_38661032.mp3,21d1c32a690ed9a5305049627c07701226c4880a9e02db...,female_feminine,lt
101,cb07d781da89d6a623a37c56addb82b9dfc8fd7ced0339...,common_voice_lt_38726139.mp3,234625b9eb49d522fa073ba4b76583834967ecdc645903...,female_feminine,lt
102,cb07d781da89d6a623a37c56addb82b9dfc8fd7ced0339...,common_voice_lt_38751511.mp3,2345c066c110689708ab601fd22bbe591adcb1d639e59d...,female_feminine,lt
103,cb07d781da89d6a623a37c56addb82b9dfc8fd7ced0339...,common_voice_lt_39296587.mp3,235ac923087ec86cdfd061afe10bed9aa0639216349715...,female_feminine,lt
698,a515a8c82315570e3f5545b2eea5aa0465956e46ad4e67...,common_voice_lt_27500874.mp3,1bebd4807eef08d36117ccc93ae5fc8113964673c76a77...,female_feminine,lt
...,...,...,...,...,...
15549,3a79f164a09807bdeee7d660c3b2d76d692ad1ff57246a...,common_voice_lt_37151639.mp3,1ef9701235800a049dac59735c3500084bdd990b475470...,female_feminine,lt
15550,3a79f164a09807bdeee7d660c3b2d76d692ad1ff57246a...,common_voice_lt_37151641.mp3,22ff5f79f44d032e01f928bb4b80f704c6fff526da1e24...,female_feminine,lt
15551,3a79f164a09807bdeee7d660c3b2d76d692ad1ff57246a...,common_voice_lt_37151642.mp3,220206e6a7825c0cc6d0693f80271eaedc4ffa0bcdb5b5...,female_feminine,lt
15552,3a79f164a09807bdeee7d660c3b2d76d692ad1ff57246a...,common_voice_lt_37151644.mp3,22dd83f76fc004685044a40fae5ae9821ed8316f2c4497...,female_feminine,lt


In [18]:
df_men = df[man_filter]
df_men

,client_id,path,sentence_id,gender,locale
22,77942f561de7e977fe0b4e02d2e519d8163be890fe4131...,common_voice_lt_37469401.mp3,207751808ca5b0f52e111e57fc28271622b218e4702ae9...,male_masculine,lt
23,77942f561de7e977fe0b4e02d2e519d8163be890fe4131...,common_voice_lt_37469402.mp3,22cb2c75e3721648d4a88ce3a4140410984acb061d9501...,male_masculine,lt
24,77942f561de7e977fe0b4e02d2e519d8163be890fe4131...,common_voice_lt_37469406.mp3,226ef0cd92c7c885009f94f8430c26f3043ee1bc9099cc...,male_masculine,lt
25,8919c0b12315e6820dbe781233b7874dfbb97f2e6b7476...,common_voice_lt_38229315.mp3,230dfcc19cf36a96901d00ed557f83302b1be803aadf59...,male_masculine,lt
26,8919c0b12315e6820dbe781233b7874dfbb97f2e6b7476...,common_voice_lt_38229318.mp3,23375a307e5ecbcbd7361d6e9f961323101a07bc29a85c...,male_masculine,lt
...,...,...,...,...,...
16638,e34db1479bd223145489573ff58e246c8b30e9de09bdc5...,common_voice_lt_25162613.mp3,10ef89654fb72a7b7b6d45deb809fb71216208a3f0f1ed...,male_masculine,lt
16639,e34db1479bd223145489573ff58e246c8b30e9de09bdc5...,common_voice_lt_25162614.mp3,10db81bb49e598d5e22d2c8fafea545691d93d08fa25cf...,male_masculine,lt
16640,e34db1479bd223145489573ff58e246c8b30e9de09bdc5...,common_voice_lt_25162615.mp3,1168759dfcaf33a91d6a0afbe2379542c1c2e5f04d4d42...,male_masculine,lt
16641,e34db1479bd223145489573ff58e246c8b30e9de09bdc5...,common_voice_lt_25162616.mp3,1141867eb08255f5a2098121972fa6b3c8b3bc9b1807eb...,male_masculine,lt


## Set number of probes in my set and split it to train, validate and test

In [ ]:
SET_SIZE = 3_000
MAX_NUMBER_OF_CLIENT_ID = 3000
MIN_CLIP_DURATION = 4000 #  clip time in [ms]

TRAIN_SIZE = int(SET_SIZE * 0.6)
VAL_SIZE = int((SET_SIZE - TRAIN_SIZE) // 2)
TEST_SIZE = SET_SIZE - TRAIN_SIZE - VAL_SIZE

print(f'Train size: {TRAIN_SIZE}')
print(f'Validation size: {VAL_SIZE}')
print(f'Test size: {TEST_SIZE}')
print(f'Sum of sizes: {TRAIN_SIZE + VAL_SIZE + TEST_SIZE} is equal to SET_SIZE {TRAIN_SIZE + VAL_SIZE + TEST_SIZE == SET_SIZE}')

In [ ]:
# Select probes with min that time 
df_clips_duration = pd.read_csv(os.path.join('..','..', 'languages', 'en', 'clip_durations.tsv'), sep='\t')
df_clips_duration['clip'] = df_clips_duration['clip']
df_men = df_men.merge(df_clips_duration, left_on='path', right_on='clip')
df_men

In [ ]:

# 
# for i in range(len(df_clips_duration)):
#     print(i)
#     row_in_df = df_men[df_men['path']== df_clips_duration.iloc[i]['clip']] 
#     if (df_clips_duration.iloc[i]['duration[ms]'] >= MIN_CLIP_DURATION) and (len(row_in_df) >= 0):
#         rows_over_min_dur_time = pd.concat([rows_over_min_dur_time, row_in_df])
#     else:
#         rows_under_min_dur_time = pd.concat([rows_under_min_dur_time, row_in_df])
# 
# 

rows_over_min_dur_time = df_men[df_men['duration[ms]'] >= MIN_CLIP_DURATION]
rows_under_min_dur_time = df_men[df_men['duration[ms]'] < MIN_CLIP_DURATION]
rows_over_min_dur_time

In [ ]:
rows_under_min_dur_time

In [ ]:
if len(rows_over_min_dur_time) >= SET_SIZE:
    df_men = rows_over_min_dur_time
else:
    df_men = pd.concat([rows_over_min_dur_time, rows_under_min_dur_time], ignore_index=True)[:SET_SIZE]

df_men

In [ ]:
df_train = pd.DataFrame()
df_val = pd.DataFrame()
df_test = pd.DataFrame()

In [ ]:
df_counted_id = df_men['client_id'].value_counts(ascending=True)
df_counted_id

In [ ]:
clients_form_origin_df = df_men[df_men['client_id'] == df_counted_id.index[200]]
clients_form_origin_df1 = df_men[df_men['client_id'] == df_counted_id.index[201]]
clients_form_origin_df1['client_id'].iloc[0]

In [ ]:
for i in range(len(df_counted_id)):
    print(df_counted_id.index[i], df_counted_id.iloc[i])
    rows_form_origin_df = df_men[df_men['client_id'] == df_counted_id.index[i]][:MAX_NUMBER_OF_CLIENT_ID]
    if len(rows_form_origin_df) <= (TEST_SIZE - len(df_test)) and rows_form_origin_df['client_id'].iloc[0]:
        df_test = pd.concat([df_test, rows_form_origin_df], ignore_index=True)
        continue
    if len(df_test) < TEST_SIZE and rows_form_origin_df['client_id'].iloc[0]:
        df_test = pd.concat([df_test, rows_form_origin_df[:(TEST_SIZE - len(df_test))]], ignore_index=True)
        continue
    if len(rows_form_origin_df) <= (VAL_SIZE - len(df_val)):
        df_val = pd.concat([df_val, rows_form_origin_df], ignore_index=True)
        continue
    if len(df_val) < VAL_SIZE and rows_form_origin_df['client_id'].iloc[0]:
        df_val = pd.concat([df_val, rows_form_origin_df[:(VAL_SIZE - len(df_val))]], ignore_index=True)
        continue
    if len(rows_form_origin_df) <= (TRAIN_SIZE - len(df_train)):
        df_train = pd.concat([df_train, rows_form_origin_df], ignore_index=True)
        continue
    

In [ ]:
df_test

In [ ]:
df_val

In [ ]:
df_train

In [ ]:
df_train['client_id'].value_counts(ascending=False)

In [ ]:
df_val['client_id'].value_counts(ascending=False)

In [ ]:
df_test['client_id'].value_counts(ascending=False)

## Audio Preprocessing

In [ ]:
df_test_filenames = df_test['path']
df_val_filenames = df_val['path']
df_train_filenames = df_train['path']
df_test_filenames

In [ ]:
df_val_filenames

In [ ]:
PL_DIR = os.path.join('..','..', 'languages', 'pl', 'clips')
PL_DIR

In [ ]:
# TEST
df_train_filenames = df_train_filenames.apply(lambda fn: os.path.join(PL_DIR, fn))
df_val_filenames = df_val_filenames.apply(lambda fn: os.path.join(PL_DIR, fn))
df_test_filenames = df_test_filenames.apply(lambda fn: os.path.join(PL_DIR, fn))
df_val_filenames

In [ ]:
# STARE
# pl_dataset_train = tf.data.Dataset.from_tensor_slices(df_train_filenames.apply(lambda fn: os.path.join(PL_DIR, fn)))
# pl_dataset_val = tf.data.Dataset.from_tensor_slices(df_val_filenames.apply(lambda fn: os.path.join(PL_DIR, fn)))
# pl_dataset_test = tf.data.Dataset.from_tensor_slices(df_test_filenames.apply(lambda fn: os.path.join(PL_DIR, fn)))



In [ ]:
sample_rate = 48_000

def add_zeros(wav, sample_rate):
    time_probes = wav.shape[0]
    missing_probes_one_side = int((MIN_CLIP_DURATION/1000* sample_rate - time_probes)//2)
    padded_tensor = tf.pad(wav.numpy(), [[missing_probes_one_side, missing_probes_one_side]])
    return tf.convert_to_tensor(padded_tensor, dtype=tf.float32)

def cut_wav(wav, sample_rate):
    time_probes = wav.shape[0]
    # clip_dur_in_sec = time_probes / sample_rate
    overlap = int((time_probes - (MIN_CLIP_DURATION/1000) * sample_rate)/2) 
    cut_clip = wav[overlap:(time_probes - overlap)]
    return tf.convert_to_tensor(cut_clip, dtype=tf.float32)

def load_wav_16k_mono_and_resample(filename, fin_sam_rate=16_000):
    file_content = tf.io.read_file(filename)
    wav = tfio.audio.decode_mp3(file_content)
    wav = tf.squeeze(wav, axis=-1)
    # sample_rate = tf.cast(sample_rate, dtype=tf.int32)
    # wav = librosa.resample(wav.numpy(), orig_sr=sample_rate.numpy(), target_sr=fin_sam_rate)
    # return wav
    # return tf.convert_to_tensor(wav, dtype=tf.float32), sample_rate.numpy()
    return tf.convert_to_tensor(wav, dtype=tf.float32)

def load_and_align_probes(file_path):
    wav = load_wav_16k_mono_and_resample(file_path)
    expected_probes = int((MIN_CLIP_DURATION/1000) * sample_rate)
    print(expected_probes)
    current_probes = wav.shape[0]
    print(current_probes)
    if expected_probes > current_probes:
        print("Add zeros")
        return add_zeros(wav, sample_rate)
    elif expected_probes < current_probes:
        print("Cut wav")
        return cut_wav(wav, sample_rate)
    return tf.convert_to_tensor(wav, dtype=tf.float32)



In [ ]:
train_align_dataset = tf.data.Dataset.from_tensor_slices(df_train_filenames.apply(lambda filename: load_and_align_probes(filename)).to_list())
test_align_dataset = tf.data.Dataset.from_tensor_slices(df_test_filenames.apply(lambda filename: load_and_align_probes(filename)).to_list())
val_align_dataset = tf.data.Dataset.from_tensor_slices(df_val_filenames.apply(lambda filename: load_and_align_probes(filename)).to_list())

test_align_dataset

In [ ]:
train_align_dataset.as_numpy_iterator().next()

In [ ]:
def align_probes(wav, sample_rate):
    # wav = load_wav_16k_mono_and_resample(file_path)
    expected_probes = (MIN_CLIP_DURATION/1000) * sample_rate
    print(expected_probes)
    current_probes = wav.shape[0]
    print(current_probes)
    if expected_probes > current_probes:
        print("Add zeros")
        return add_zeros(wav, sample_rate)
    if expected_probes < current_probes:
        print("Cut wav")
        return cut_wav(wav, sample_rate)
    return wav
    

In [ ]:
MIN_CLIP_DURATION/1000* sample_rate

In [ ]:
def increase_amplitude(wav, min_increase=2.0, max_increase=5.0):
    increased_wav = wav * random.uniform(min_increase, max_increase)
    return tf.convert_to_tensor(increased_wav, dtype=tf.float32)

In [ ]:
def normalize_audio(wav):
    max_amplitude = tf.reduce_max(tf.abs(wav))
    normalized_wav = wav / max_amplitude  # Normalizacja do zakresu [-1, 1]
    return normalized_wav

In [ ]:
def add_noise(wav, noise_level=0.1):
    noise = tf.random.normal(tf.shape(wav), mean=0.0, stddev=noise_level)
    return wav + noise

In [ ]:
def time_masking(wav, max_mask_length=10000):
    # Sprawdzenie długości wav
        # Sprawdzenie czy wav jest pusty lub ma nieprawidłowy kształt
    if wav is None or tf.rank(wav) != 1:
        return wav
    
    # Sprawdzenie długości wav
    if tf.shape(wav)[0] <= max_mask_length:
        return wav
    
    # Losowa długość maskowania
    mask_length = tf.random.uniform([], maxval=max_mask_length, dtype=tf.int32)
    
    # Sprawdzenie, czy maska nie wyjdzie poza zakres
    mask_start_max = tf.shape(wav)[0] - mask_length
    mask_start = tf.random.uniform([], maxval=mask_start_max, dtype=tf.int32)
    
    # Stworzenie maski czasowej
    mask = tf.concat([
        tf.ones([mask_start]),
        tf.zeros([mask_length]),
        tf.ones([tf.shape(wav)[0] - mask_start - mask_length])
    ], axis=0)
    
    # Zastosowanie maskowania do sygnału audio
    masked_wav = wav * mask
    
    return masked_wav


In [ ]:
def change_pitch(wav, sample_rate=48000, pitch_shift=2):
    wav_np = wav.numpy()  # Convert tensor to numpy array
    pitched_wav = librosa.effects.pitch_shift(wav_np, sr=sample_rate, n_steps=pitch_shift)
    return tf.convert_to_tensor(pitched_wav, dtype=tf.float32)


In [ ]:
def speed_up_audio(wav, speed_factor=2):
    wav_np = wav.numpy()
    stretched_wav = librosa.effects.time_stretch(wav_np, rate=speed_factor)   
    return stretched_wav

In [ ]:
def slow_down_audio(wav, speed_factor=0.5):
    wav_np = wav.numpy()
    slowed_wav_np = librosa.effects.time_stretch(wav_np, rate=speed_factor)
    return tf.convert_to_tensor(slowed_wav_np, dtype=tf.float32)


In [ ]:

audio_processing_functions = [increase_amplitude, add_noise, time_masking]
def process_random_samples(dataset, num_samples_to_process):
    processed_samples = []
    shuffled_dataset = dataset.shuffle(buffer_size=num_samples_to_process)
    samples =  shuffled_dataset.take(num_samples_to_process)
    for sample in samples:
        processing_function = random.choice(audio_processing_functions)
        processed_sample = processing_function(sample)
    
        processed_samples.append(processed_sample)

    return processed_samples

In [ ]:
processed_samples_train = process_random_samples(train_align_dataset, TRAIN_SIZE - len(train_align_dataset))
# processed_samples_val = process_random_samples(val_align_dataset, (VAL_SIZE - len(val_align_dataset)))
# processed_samples_test = process_random_samples(test_align_dataset, (TEST_SIZE - len(test_align_dataset)))
# processed_samples_test

In [ ]:
align_processed_train = [align_probes(tensor, sample_rate) for tensor in processed_samples_train]
# align_processed_val = [align_probes(tensor, sample_rate) for tensor in processed_samples_val]
# align_processed_test = [align_probes(tensor,sample_rate) for tensor in processed_samples_test]
align_processed_train

In [ ]:
processed_samples_train_dataset = tf.data.Dataset.from_tensor_slices(align_processed_train)
# processed_samples_val_dataset = tf.data.Dataset.from_tensor_slices(align_processed_val)
# processed_samples_test_dataset = tf.data.Dataset.from_tensor_slices(align_processed_test)

In [ ]:

train_dataset_con = train_align_dataset.concatenate(processed_samples_train_dataset)
# val_dataset_con = val_align_dataset.concatenate(processed_samples_val_dataset)
# test_dataset_con = test_align_dataset.concatenate(processed_samples_test_dataset)


In [ ]:
align_pl_dataset_train_with_processed_samples_norm = train_dataset_con.map(lambda audio: normalize_audio(audio))
align_pl_dataset_val_with_processed_samples_norm = val_align_dataset.map(lambda audio: normalize_audio(audio))
align_pl_dataset_test_with_processed_samples_norm = test_align_dataset.map(lambda audio: normalize_audio(audio))

In [ ]:
def create_spectrogram(wav):
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram

In [ ]:
spectrogram_pl_dataset_train = align_pl_dataset_train_with_processed_samples_norm.map(lambda audio: create_spectrogram(audio))
spectrogram_pl_dataset_val = align_pl_dataset_val_with_processed_samples_norm.map(lambda audio: create_spectrogram(audio))
spectrogram_pl_dataset_test = align_pl_dataset_test_with_processed_samples_norm.map(lambda audio: create_spectrogram(audio))

In [ ]:
spectrogram_pl_dataset_train

In [ ]:
spectrogram_pl_dataset_val

In [ ]:
spectrogram_pl_dataset_test

In [ ]:
spectrogram_pl_dataset_test.as_numpy_iterator().next().shape

In [ ]:
languages = ['pl', 'en', 'ls']
language_to_index = {lang: idx for idx, lang in enumerate(languages)}
print(language_to_index)

def one_hot_encode_language(lang):
    lang_index = language_to_index[lang]
    one_hot = tf.one_hot(lang_index, len(languages))
    return one_hot

one_hot_encode_language('en')

In [ ]:
labeled_train_dataset = tf.data.Dataset.zip((spectrogram_pl_dataset_train, tf.data.Dataset.from_tensor_slices([one_hot_encode_language('en')] * len(spectrogram_pl_dataset_train))))
labeled_val_dataset = tf.data.Dataset.zip((spectrogram_pl_dataset_val, tf.data.Dataset.from_tensor_slices(tf.ones(len(spectrogram_pl_dataset_val)))))
labeled_test_dataset = tf.data.Dataset.zip((spectrogram_pl_dataset_test, tf.data.Dataset.from_tensor_slices(tf.ones(len(spectrogram_pl_dataset_test)))))

In [ ]:
labeled_train_dataset.as_numpy_iterator().next()

In [ ]:
labeled_val_dataset.as_numpy_iterator().next()

In [ ]:
labeled_test_dataset.as_numpy_iterator().next()